# How to build Angular components?

How to create a mock filesystem?


Run angular-cli commands from nodejs?

Add a component to the specified project using angular-cli?



In [ ]:
var execSync = require('child_process').execSync;
try {
    require.resolve('@angular/cli');
} catch (e) {
    execSync('npm install @angular/cli');
}

// use utility/filesystem to mock all fs and typescript commands
var mockTypescriptFs, cli;

var importer = require('../Core');
// call the CLI just like from command line
var ng = (project, args = ['generate', 'component', 'test']) => {
    // set up project path
    var previous = process.cwd();
    
    var conf = {
        cliArgs: args,
        inputStream: null,
        outputStream: process.stdout
    };
    
    // execute
    return importer.interpretAll('memory-fs rewire')
        .then(r => {
            if(typeof mockTypescriptFs == 'undefined') {
                mockTypescriptFs = eval("'use strict';" + r[0].code);
                // overlay out temp filesystem on top of current filesystem
                mockTypescriptFs(project);
                cli = require('@angular/cli');
            }
            process.chdir(project);
            return cli(conf);
        });
};
ng;



Compile the project using webpack?

In [ ]:
var execSync = require('child_process').execSync;
    try {
    require.resolve('webpack');
} catch (e) {
    execSync('npm install webpack');
}

// source https://github.com/christianalfoni/webpack-bin/issues/106
var webpackAngularProject = (project) => {
    // set up project path
    var previous = process.cwd();
    process.chdir(project);
    module.paths.unshift(project + '/node_modules');
    
    var webpack = require('webpack');
    var webpackConfig = require(path.join(project, 'webpack.config.js'));
    var gulp = require(path.join(project, 'config', 'gulp-helpers.js'));
    var settings = gulp.loadSettings(require(path.join(project, 'config', 'build-config.json')));
    var config = webpackConfig({
          env: 'prod',
          platform: 'server'
        }, gulp.root, settings);
    var compiler = webpack(config);

    // TODO: do we need this since it is already mocked in angular-cli
    //mockTypescriptFs = eval("'use strict';" + r[0].code);
    //mockTypescriptFs(project, data);
    //compiler.inputFileSystem = memfs;
    //compiler.outputFileSystem = memfs;
    //compiler.resolvers.normal.fileSystem = memfs;
    //compiler.resolvers.context.fileSystem = memfs;

    return new Promise((resolve, reject) => {
        compiler.run(function(err, stats) {
            process.chdir(previous);
            if (err) return reject(err);
            resolve(stats.compilation.assets['server.js'].source());
        });
    });
};
webpackAngularProject;


In [ ]:

//ng(path.dirname(project), ['new', '--skip-git', '--skip-install', 'new-portal'])
//    .then(() => ng(project, [
//    'generate', 'module', '--spec', '--flat', 'AppServer']))
    /*
    //.then(() => npm(project, ['install'], {'save': true, bin-links': false, verbose: true, prefix: project}))
    //.then(() => npm(project, ['install', '@angular/material', '@angular/cdk'], {'save': true, bin-links': false, verbose: true, prefix: project}))
    //.then(() => npm(project, ['list'], {prefix: project}))
    //.then(() => npm(project, ['prefix'], {prefix: project}))
    //convertNgUniversal(project)
    //    .then(() => ng(project, ['build', '--aot', '--prod']))
    //.then(() => webpackAngularProject(project))
    .then(r => {
        process.chdir(project);
        var server = require(path.join(project, '.server', 'server.js'));
        console.log(server);
    })
    */

// TODO:
// build the project in memory
// make the sockify server send us logs
// accept logins and encrypt in to password file?
// send mock responses to display all interfaces?  e.g. logging in displays redirect and error "you are already logged in" on login page at the same time


Set angular theme colors?